In [6]:
!apt-get update -qq
!apt-get install -y bwa samtools bcftools


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bcftools is already the newest version (1.13-1).
bwa is already the newest version (0.7.17-6).
samtools is already the newest version (1.13-4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [7]:
from google.colab import files

uploaded = files.upload()

Saving sample.fasta to sample (1).fasta


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
import subprocess
import tempfile
from pathlib import Path
import shutil
input_fasta = "/content/sample.fasta"
reference = "/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa"
output_vcf = "variants.vcf"
threads = "2"

if not Path(f"{reference}.bwt").exists():
    bwa_path = shutil.which("bwa")
    if bwa_path is None:
        raise RuntimeError("bwa not found in PATH. Please install or add it to your PATH.")


    if not os.path.exists(reference):
        raise FileNotFoundError(f"Reference file '{reference}' not found.")


    subprocess.run([bwa_path, "index", reference], check=True)

if not Path(f"{reference}.fai").exists():
    subprocess.run(["samtools", "faidx", reference], check=True)

with tempfile.TemporaryDirectory() as tmp:
    sam = os.path.join(tmp, "aln.sam")
    bam = os.path.join(tmp, "aln.bam")
    sorted_bam = os.path.join(tmp, "aln.sorted.bam")
    raw_vcf = os.path.join(tmp, "raw.vcf")
    with open(sam, 'w') as sam_out:
        subprocess.run(["bwa", "mem", "-t", threads, reference, input_fasta],
                       stdout=sam_out, check=True)

    # SAM -> BAM
    subprocess.run(["samtools", "view", "-bS", sam, "-o", bam], check=True)

    # Sort BAM
    subprocess.run(["samtools", "sort", "-@", threads, "-o", sorted_bam, bam], check=True)

    # Index BAM
    subprocess.run(["samtools", "index", sorted_bam], check=True)

    # mpileup + call (no pipes)
    with open(raw_vcf, 'w') as vcf_out:
        mpileup = subprocess.Popen(["bcftools", "mpileup", "-f", reference, sorted_bam],
                                   stdout=subprocess.PIPE)
        subprocess.run(["bcftools", "call", "-mv"], stdin=mpileup.stdout, stdout=vcf_out, check=True)

    # Normalize
    subprocess.run(["bcftools", "norm", "-f", reference, "-o", output_vcf, raw_vcf], check=True)

print(f"✅ Variant calling completed. Output: {output_vcf}")

✅ Variant calling completed. Output: variants.vcf


In [ ]:
filepath='/content/sample_fasta'
def read_fasta(filepath):
    sequence = ''
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequence += line.strip()
    return sequence


def write_vcf_header(vcf_file):
    vcf_file.write('##fileformat=VCFv4.2\n')
    vcf_file.write('##source=SimpleVariantCaller\n')
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n')

def generate_variants(reference, sample, chrom_name='chr1'):
    variants = []
    length = min(len(reference), len(sample))
    for pos in range(length):
        ref_base = reference[pos]
        alt_base = sample[pos]

        if ref_base != alt_base:
            variant = {
                'CHROM': chrom_name,
                'POS': pos + 1,
                'ID': '.',
                'REF': ref_base,
                'ALT': alt_base,
                'QUAL': '100',
                'FILTER': 'PASS',
                'INFO': '.',
                'FORMAT': 'GT',
                'SAMPLE': '0/1'
            }
            variants.append(variant)
    return variants

def save_variants_to_vcf(variants, output_path):
    with open(output_path, 'w') as vcf_file:
        write_vcf_header(vcf_file)
        for var in variants:
            row = f"{var['CHROM']}\t{var['POS']}\t{var['ID']}\t{var['REF']}\t{var['ALT']}\t{var['QUAL']}\t{var['FILTER']}\t{var['INFO']}\t{var['FORMAT']}\t{var['SAMPLE']}\n"
            vcf_file.write(row)


reference_fasta_content = """>chr1
ACTGACTGACTG
"""
sample_fasta_content = """>chr1
ACTGATTGACTA
"""

with open('reference.fasta', 'w') as f:
    f.write(reference_fasta_content)

with open('sample.fasta', 'w') as f:
    f.write(sample_fasta_content)


reference = read_fasta('reference.fasta')
sample = read_fasta('sample.fasta')

variants = generate_variants(reference, sample)
save_variants_to_vcf(variants, 'variants.vcf')


with open('variants.vcf', 'r') as file:
    vcf_content = file.read()

print(vcf_content)


In [10]:

reference_fasta_content = """>chr1
ACTGACTGACTG
"""
sample_fasta_content = """>chr1
ACTGATTGACTA
"""

with open('/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa', 'w') as f:
    f.write(reference_fasta_content)

with open('/content/sample.fasta', 'w') as f:
    f.write(sample_fasta_content)

def read_fasta(filepath):
    sequence = ''
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequence += line.strip()
    return sequence

def write_vcf_header(vcf_file):
    vcf_file.write('##fileformat=VCFv4.2\n')
    vcf_file.write('##source=SimpleVariantCaller\n')
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n')

def generate_variants(reference, sample, chrom_name='chr1'):
    variants = []
    length = min(len(reference), len(sample))
    for pos in range(length):
        ref_base = reference[pos]
        alt_base = sample[pos]

        if ref_base != alt_base:
            variant = {
                'CHROM': chrom_name,
                'POS': pos + 1,
                'ID': '.',
                'REF': ref_base,
                'ALT': alt_base,
                'QUAL': '100',
                'FILTER': 'PASS',
                'INFO': '.',
                'FORMAT': 'GT',
                'SAMPLE': '0/1'
            }
            variants.append(variant)
    return variants

def save_variants_to_vcf(variants, output_path):
    with open(output_path, 'w') as vcf_file:
        write_vcf_header(vcf_file)
        for var in variants:
            row = f"{var['CHROM']}\t{var['POS']}\t{var['ID']}\t{var['REF']}\t{var['ALT']}\t{var['QUAL']}\t{var['FILTER']}\t{var['INFO']}\t{var['FORMAT']}\t{var['SAMPLE']}\n"
            vcf_file.write(row)

reference = read_fasta('/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa')
sample = read_fasta('/content/sample.fasta')


variants = generate_variants(reference, sample)


output_path = '/content/variants.vcf'
save_variants_to_vcf(variants, output_path)

with open(output_path, 'r') as f:
    vcf_content = f.read()

print(vcf_content)


##fileformat=VCFv4.2
##source=SimpleVariantCaller
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE
chr1	6	.	C	T	100	PASS	.	GT	0/1
chr1	12	.	G	A	100	PASS	.	GT	0/1



In [14]:
import random
first_names = ['Alice', 'James', 'Maria', 'Omar', 'Lina', 'Tom', 'Sarah', 'John', 'Nina', 'David', 'Ella', 'Arjun', 'Priya', 'Liam', 'Emma']
last_names = ['Smith', 'Lee', 'Gonzalez', 'Rahman', 'Patel', 'Ray', 'Johnson', 'Khan', 'Brown', 'Singh', 'Kim', 'Chen', 'Garcia', 'White', 'Sharma']
genders = ['male', 'female']
nucleotides = ['A', 'T', 'C', 'G']

def random_dna_sequence(length):
    return ''.join(random.choices(nucleotides, k=length))


sample_data = []

for _ in range(100):
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    age = random.randint(18, 60)
    gender = random.choice(genders)
    dna_length = random.randint(10, 100)
    dna_sequence = random_dna_sequence(dna_length)

    person = {
        "name": name,
        "age": age,
        "gender": gender,
        "dna_sequence": dna_sequence
    }
    sample_data.append(person)

from pprint import pprint
pprint(sample_data[:5])
print(len(sample_data))


[{'age': 54,
  'dna_sequence': 'CTCGTCAGGC',
  'gender': 'male',
  'name': 'Ella Patel'},
 {'age': 48,
  'dna_sequence': 'AGATTCCTAGAACGATAAGCTATAGGAGCGA',
  'gender': 'male',
  'name': 'Emma Smith'},
 {'age': 28,
  'dna_sequence': 'TTAAAACGGAAAGAAGCATTTTCGGCTTCGACTTCAAACGGTATCTCCACTGTCCGGAGCGGTGGGACGATACAGCCTAGCAG',
  'gender': 'male',
  'name': 'Ella Chen'},
 {'age': 23,
  'dna_sequence': 'ACGATTGGTTTTTACTTGACAACCTGCTGTGAACTCGCAGCTGGGCGGACATCTCGCAAAGGAAGA',
  'gender': 'male',
  'name': 'David Chen'},
 {'age': 43,
  'dna_sequence': 'CTGTCTAGAGTCACATACTTTTGCTCGATCTGTTGATTTTCAGGGGTCCCCACGCCGTGT',
  'gender': 'male',
  'name': 'Emma Johnson'}]
100


In [15]:
with open("sample_new.fasta", "w") as fasta_file:
    for i, person in enumerate(sample_data):
        name_tag = person["name"].replace(" ", "_")
        fasta_file.write(f">seq{i}_{name_tag}\n{person['dna_sequence']}\n")

print(" FASTA file created: sample_new.fasta")

 FASTA file created: sample_new.fasta


In [16]:

reference_fasta_content = """>chr1
ACTGACTGACTG
"""
sample_fasta_content = """>chr1
ACTGATTGACTA
"""

with open('/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa', 'w') as f:
    f.write(reference_fasta_content)

with open('/content/sample_new.fasta', 'w') as f:
    f.write(sample_fasta_content)

def read_fasta(filepath):
    sequence = ''
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequence += line.strip()
    return sequence

def write_vcf_header(vcf_file):
    vcf_file.write('##fileformat=VCFv4.2\n')
    vcf_file.write('##source=SimpleVariantCaller\n')
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n')

def generate_variants(reference, sample, chrom_name='chr1'):
    variants = []
    length = min(len(reference), len(sample))
    for pos in range(length):
        ref_base = reference[pos]
        alt_base = sample[pos]

        if ref_base != alt_base:
            variant = {
                'CHROM': chrom_name,
                'POS': pos + 1,
                'ID': '.',
                'REF': ref_base,
                'ALT': alt_base,
                'QUAL': '100',
                'FILTER': 'PASS',
                'INFO': '.',
                'FORMAT': 'GT',
                'SAMPLE': '0/1'
            }
            variants.append(variant)
    return variants

def save_variants_to_vcf(variants, output_path):
    with open(output_path, 'w') as vcf_file:
        write_vcf_header(vcf_file)
        for var in variants:
            row = f"{var['CHROM']}\t{var['POS']}\t{var['ID']}\t{var['REF']}\t{var['ALT']}\t{var['QUAL']}\t{var['FILTER']}\t{var['INFO']}\t{var['FORMAT']}\t{var['SAMPLE']}\n"
            vcf_file.write(row)

reference = read_fasta('/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa')
sample = read_fasta('/content/sample_new.fasta')


variants = generate_variants(reference, sample)


output_path = '/content/variants.vcf'
save_variants_to_vcf(variants, output_path)

with open(output_path, 'r') as f:
    vcf_content = f.read()

print(vcf_content)


##fileformat=VCFv4.2
##source=SimpleVariantCaller
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE
chr1	6	.	C	T	100	PASS	.	GT	0/1
chr1	12	.	G	A	100	PASS	.	GT	0/1



In [17]:
def read_fasta(filepath):
    sequence = ''
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequence += line.strip()
    return sequence

def write_vcf_header(vcf_file):
    vcf_file.write('##fileformat=VCFv4.2\n')
    vcf_file.write('##source=SimpleVariantCaller\n')
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n')

def generate_variants(reference, sample, sample_name, chrom_name='chr1'):
    variants = []
    length = min(len(reference), len(sample))
    for pos in range(length):
        ref_base = reference[pos]
        alt_base = sample[pos]
        if ref_base != alt_base:
            variant = {
                'CHROM': chrom_name,
                'POS': pos + 1,
                'ID': '.',
                'REF': ref_base,
                'ALT': alt_base,
                'QUAL': '100',
                'FILTER': 'PASS',
                'INFO': '.',
                'FORMAT': 'GT',
                'SAMPLE': sample_name.replace(' ', '_')
            }
            variants.append(variant)
    return variants

def save_variants_to_vcf(variants, output_path):
    with open(output_path, 'w') as vcf_file:
        write_vcf_header(vcf_file)
        for var in variants:
            row = f"{var['CHROM']}\t{var['POS']}\t{var['ID']}\t{var['REF']}\t{var['ALT']}\t{var['QUAL']}\t{var['FILTER']}\t{var['INFO']}\t{var['FORMAT']}\t{var['SAMPLE']}\n"
            vcf_file.write(row)
reference = read_fasta('/content/drive/MyDrive/Homo_sapiens.GRCh38.dna.alt.fa')


all_variants = []

for sample in sample_data:
    sample_seq = sample['dna_sequence']
    sample_name = sample['name']
    sample_variants = generate_variants(reference, sample_seq, sample_name)
    all_variants.extend(sample_variants)

output_path = '/content/variants_all_samples.vcf'
save_variants_to_vcf(all_variants, output_path)

with open(output_path, 'r') as file:
    print(file.read())

##fileformat=VCFv4.2
##source=SimpleVariantCaller
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE
chr1	1	.	A	C	100	PASS	.	GT	Ella_Patel
chr1	2	.	C	T	100	PASS	.	GT	Ella_Patel
chr1	3	.	T	C	100	PASS	.	GT	Ella_Patel
chr1	5	.	A	T	100	PASS	.	GT	Ella_Patel
chr1	7	.	T	A	100	PASS	.	GT	Ella_Patel
chr1	9	.	A	G	100	PASS	.	GT	Ella_Patel
chr1	2	.	C	G	100	PASS	.	GT	Emma_Smith
chr1	3	.	T	A	100	PASS	.	GT	Emma_Smith
chr1	4	.	G	T	100	PASS	.	GT	Emma_Smith
chr1	5	.	A	T	100	PASS	.	GT	Emma_Smith
chr1	7	.	T	C	100	PASS	.	GT	Emma_Smith
chr1	8	.	G	T	100	PASS	.	GT	Emma_Smith
chr1	10	.	C	G	100	PASS	.	GT	Emma_Smith
chr1	11	.	T	A	100	PASS	.	GT	Emma_Smith
chr1	12	.	G	A	100	PASS	.	GT	Emma_Smith
chr1	1	.	A	T	100	PASS	.	GT	Ella_Chen
chr1	2	.	C	T	100	PASS	.	GT	Ella_Chen
chr1	3	.	T	A	100	PASS	.	GT	Ella_Chen
chr1	4	.	G	A	100	PASS	.	GT	Ella_Chen
chr1	6	.	C	A	100	PASS	.	GT	Ella_Chen
chr1	7	.	T	C	100	PASS	.	GT	Ella_Chen
chr1	9	.	A	G	100	PASS	.	GT	Ella_Chen
chr1	10	.	C	A	100	PASS	.	GT	Ella_Chen
chr1	11	.	T	A	100	PASS	.	GT